### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, we will:

- Run a hyperparameter sweep on a training script
- Compare the results of the runs in the MLflow UI
- choose the best run and register it as a model
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform

In [51]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [52]:
## load the dataset
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [53]:
## Split the data into training, validation and test sets

train, test = train_test_split(data, test_size=0.25, random_state=42)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2835,6.3,0.25,0.22,3.30,0.048,41.0,161.0,0.99256,3.16,0.50,10.5,6
1157,7.8,0.30,0.29,16.85,0.054,23.0,135.0,0.99980,3.16,0.38,9.0,6
744,7.4,0.38,0.27,7.50,0.041,24.0,160.0,0.99535,3.17,0.43,10.0,5
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,6
3338,7.2,0.27,0.28,15.20,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [54]:
train_x=train.drop(["quality"], axis=1).values
train_y=train[["quality"]].values.ravel()

## test dataset
test_x=test.drop(['quality'], axis=1).values
test_y=test["quality"].values.ravel()

## Splitting this train into train and validation data
train_x,valid_x,train_y,valid_y=train_test_split(train_x, train_y, test_size=0.20, random_state=42)

signature = infer_signature(train_x, train_y)

In [55]:
### ANN Model

def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    ## Define model architecture
    mean=np.mean(train_x, axis=0)
    var=np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"], momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLflow tracking

    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y),
                  epochs=epochs, batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x, valid_y, batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## Log the model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}



In [56]:
def objective(params):
    ## MLflow will track the parameters and results for each run

    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [57]:
space={
    "lr":hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum":hp.uniform("momentum", 0.0, 1.0)
}
space

{'lr': <hyperopt.pyll.base.Apply at 0x7cfd6273fe60>,
 'momentum': <hyperopt.pyll.base.Apply at 0x7cfd6273f980>}

In [58]:
mlflow.set_experiment("/wine-quality")

with mlflow.start_run():

    #Conduct the hyperparameter search using Hyperopt

    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )


    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")





Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1:13 2s/step - loss: 34.9241 - root_mean_squared_error: 5.9097
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.3950 - root_mean_squared_error: 2.7192  
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2.2927 - root_mean_squared_error: 1.5142 - val_loss: 0.8054 - val_root_mean_squared_error: 0.8974

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 1.0789 - root_mean_squared_error: 1.0387
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8184 - root_mean_squared_error: 0.9030 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6730 - root_mean_squared_error: 0.8204 - val_loss: 0.5982 - val_root_mean_squared_error: 0.7734

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 0.5832 - root_mean_squared_error: 0.7637
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5790 - root_mean_squared_error: 0.7609 
46/46 ━

2025/11/13 16:12:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 27s 605ms/step - loss: 36.4122 - root_mean_squared_error: 6.0343
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.2187 - root_mean_squared_error: 4.6487   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 7.7316 - root_mean_squared_error: 2.7806 - val_loss: 1.9206 - val_root_mean_squared_error: 1.3859

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 1.4454 - root_mean_squared_error: 1.2023
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5479 - root_mean_squared_error: 1.2438 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5506 - root_mean_squared_error: 1.2453 - val_loss: 1.4743 - val_root_mean_squared_error: 1.2142

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 1.1951 - root_mean_squared_error: 1.0932
21/46 ━━━━━━━━

2025/11/13 16:12:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 24s 535ms/step - loss: 32.9266 - root_mean_squared_error: 5.7382
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.0778 - root_mean_squared_error: 4.7686   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.6087 - root_mean_squared_error: 3.0998 - val_loss: 2.1755 - val_root_mean_squared_error: 1.4750

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 2.2824 - root_mean_squared_error: 1.5108
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8034 - root_mean_squared_error: 1.3423 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.7482 - root_mean_squared_error: 1.3222 - val_loss: 1.5992 - val_root_mean_squared_error: 1.2646

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 1.7075 - root_mean_squared_error: 1.3067
25/46 ━━━━━━━━

...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 26s 588ms/step - loss: 34.9206 - root_mean_squared_error: 5.9094
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.2843 - root_mean_squared_error: 5.8552   
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 32.8953 - root_mean_squared_error: 5.7354 - val_loss: 30.3399 - val_root_mean_squared_error: 5.5082

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 31.0377 - root_mean_squared_error: 5.5711
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.3992 - root_mean_squared_error: 5.4218 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 27.4770 - root_mean_squared_error: 5.2419 - val_loss: 25.2786 - val_root_mean_squared_error: 5.0278

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 24.9647 - root_mean_squared_error: 4.9965
25/46 ━

2025/11/13 16:13:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:49<00:00, 12.41s/trial, best loss: 0.7311812043190002]

2025/11/13 16:13:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.050757766999646375), 'momentum': np.float64(0.1642935501366477)}
Best eval rmse: 0.7311812043190002
